# Chapter 7 — Multi-Agent: Coordinator

A **coordinator agent** dynamically delegates tasks to specialist sub-agents based on the request.

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

print("Google API Key set:", bool(os.environ.get("GOOGLE_API_KEY")))

In [ ]:
from google.adk.agents import Agent

# Specialist agents
coding_agent = Agent(
    name="coding_expert",
    model="gemini-2.0-flash",
    description="Handles programming, coding, and software development questions.",
    instruction="You are an expert software developer. Help with coding questions, debug issues, explain algorithms, and write code snippets.",
)

writing_agent = Agent(
    name="writing_expert",
    model="gemini-2.0-flash",
    description="Handles writing, editing, and content creation tasks.",
    instruction="You are an expert writer and editor. Help with writing tasks, improve text quality, and create compelling content.",
)

math_agent = Agent(
    name="math_expert",
    model="gemini-2.0-flash",
    description="Handles mathematics, statistics, and data analysis questions.",
    instruction="You are a mathematics expert. Solve math problems step by step, explain concepts clearly, and help with statistical analysis.",
)

# Coordinator
coordinator = Agent(
    name="coordinator",
    model="gemini-2.0-flash",
    description="Routes requests to the best specialist.",
    instruction="""You are a coordinator agent. Analyze each user request and delegate it
to the most appropriate specialist:
- coding_expert: for programming and software questions
- writing_expert: for writing and content tasks  
- math_expert: for math and data analysis

Route to the specialist whose expertise best matches the request.""",
    sub_agents=[coding_agent, writing_agent, math_agent],
)

print("Coordinator ready with specialists:", [a.name for a in coordinator.sub_agents])

In [ ]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

session_service = InMemorySessionService()
runner = Runner(agent=coordinator, app_name="coord_demo", session_service=session_service)
session = await session_service.create_session(app_name="coord_demo", user_id="user1")

# Test with different types of requests
requests = [
    "Write a Python function to reverse a linked list.",
    "Write a compelling product description for a smart water bottle.",
    "What is the derivative of x^3 * sin(x)?",
]

for text in requests:
    print(f"\n{'='*60}")
    print(f"USER: {text}")
    print(f"{'='*60}")
    
    msg = types.Content(role="user", parts=[types.Part(text=text)])
    response = await runner.run_async(user_id="user1", session_id=session.id, new_message=msg)
    
    for event in response:
        if event.content and event.content.parts:
            print(f"[{event.author}]: {event.content.parts[0].text[:200]}...")

## Key Takeaways

- The coordinator uses sub-agent `description` fields to decide routing
- ADK's Auto-Flow handles the delegation logic automatically
- Scales easily — just add more specialist sub-agents